In [0]:
import pandas as pd
import glob
num_data = 4
path = r'/content'

In [0]:
path = r'/Users/juho/Downloads/Outdoor'
acc, gps, rss, label = [], [], [], []

sensors = ['Accelerometer', 'Real_GPS', 'Rss', 'Transition']

for sensor in sensors:
    all_files = glob.glob(f'{path}/*/{sensor}.csv')
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        print(df.shape)
        if sensor == 'Accelerometer':
            acc.append(df)
        elif sensor == 'Real_GPS':
            gps.append(df)
        elif sensor == 'Rss':
            rss.append(df)
        elif sensor == 'Transition':
            label.append(df)

### Convert date to unix timestamp

In [0]:
from datetime import datetime
GPS = []
for g in gps:
    GPS.append(g.assign(timeMillis = lambda x: x['time'].apply(lambda y: (datetime.strptime(y, "%Y-%m-%dT%H:%M:%S.%fZ").timestamp() + 32400) * 1000)))

### Select data after starting point (e.g. press -1 button)

In [0]:
for i, l in enumerate(label):
    startTime = l['timeMillis'][0]
    print(rss[i].shape)
    rss[i] = rss[i].loc[rss[i]['timeMillis']  > startTime]
    print(rss[i].shape)
    print(acc[i].shape)
    acc[i] = acc[i].loc[acc[i]['timeMillis']  > startTime]
    print(acc[i].shape)

(53354, 6)
(53354, 6)
(339378, 5)
(339378, 5)
(13306, 6)
(13306, 6)
(74992, 5)
(74992, 5)
(27825, 6)
(27825, 6)
(267889, 5)
(267889, 5)
(25510, 6)
(25510, 6)
(158980, 5)
(158980, 5)


### Select 5GHZ bandwidth

In [0]:
RSS_5GHZ = []
for r in rss:
    print(r.shape)
    RSS_5GHZ.append(r.loc[(r['frequency'] >= 5000) & (r['frequency'] < 6000)])
for r in RSS_5GHZ:
    print(r.shape)

(53354, 6)
(13306, 6)
(27825, 6)
(25510, 6)
(32482, 6)
(5974, 6)
(14560, 6)
(12583, 6)


### Select 2GHZ bandwidth

In [0]:
RSS_2GHZ = []
for r in rss:
    print(r.shape)
    RSS_2GHZ.append(r.loc[(r['frequency'] >= 2000) & (r['frequency'] < 3000)])
for r in RSS_2GHZ:
    print(r.shape)

(53354, 6)
(13306, 6)
(27825, 6)
(25510, 6)
(20872, 6)
(7332, 6)
(13265, 6)
(12927, 6)


In [0]:
print(acc[0]['timeMillis'].min())

1574385249607


In [0]:
start = []
for i in range(4):
    min_arr = [RSS_2GHZ[i]['timeMillis'].min(), RSS_5GHZ[i]['timeMillis'].min(), GPS[i]['timeMillis'].min(), acc[i]['timeMillis'].min()]
    start.append(max(min_arr))
end = []
for i in range(4):
    max_arr = [RSS_2GHZ[i]['timeMillis'].max(), RSS_5GHZ[i]['timeMillis'].max(), GPS[i]['timeMillis'].max(), acc[i]['timeMillis'].max()]
    end.append(min(max_arr))

In [0]:
print(start)
print(end)

[1574385251822, 1574764503287, 1574998672996.0, 1575247525741]
[1574388640923, 1574765894850, 1575001186231, 1575250157784]


In [0]:
GPS[0].head()['timeMillis'][0]

1574385242474.0

### Lowpass Filter (Moving Average)

In [0]:
from scipy import signal
import numpy as np
from pylab import *

def ma(x, n = 20):
    b = np.repeat(1.0/n, n) #Create impulse response
    xf = signal.lfilter(b, 1, x) #Filter the signal
    return(xf)

In [0]:
featured_rss = []
WIN_SIZE_IN_MS = 1000 
OVERLAP_RATIO = 0.5 

for i in range(4):
    START_TIME, END_TIME = start[i], end[i]
    
    # arange([start, ]stop, [step, ]dtype=None)
    # find the end time of each window by considering the overlapping ratio 
    WINDOWS = np.arange(START_TIME + WIN_SIZE_IN_MS, END_TIME, WIN_SIZE_IN_MS * (1 - OVERLAP_RATIO))

    # new dataframe 
    FEATURES_TIME = []
    for w in WINDOWS:
        # for a given window, set the start and end time stamps 
        win_start, win_end = w - WIN_SIZE_IN_MS, w
      
        for var in ['rss']:
            # select the rows that belong to the current window, w
            value_5GHZ = RSS_5GHZ[i].loc[
                lambda x: (x['timeMillis'] >= win_start) & (x['timeMillis'] < win_end), var
            ].values
            value_2GHZ = RSS_2GHZ[i].loc[
                lambda x: (x['timeMillis'] >= win_start) & (x['timeMillis'] < win_end), var
            ].values
            # extract basic features 
            #min_v = np.min(value) # min
            #max_v = np.max(value) # max
            #mean_v = np.mean(value) # mean
            #std_v = np.std(value) # std. dev.

            # append each result (w: current window's end-timestamp, extracted feature) as a new row
            #FEATURES_TIME.append((w - , '{}-{}'.format('Min', var), min_v))
            #FEATURES_TIME.append((w, '{}-{}'.format('Max', var), max_v))
            #FEATURES_TIME.append((w, '{}-{}'.format('Mean', var), mean_v))
            #FEATURES_TIME.append((w, '{}-{}'.format('Std', var), std_v))
            if (value_5GHZ.size > 0):
                FEATURES_TIME.append((w, f'{var}-5GHZ-MEAN', np.mean(value_5GHZ)))
                FEATURES_TIME.append((w, f'{var}-5GHZ-MAX', np.max(value_5GHZ)))
                FEATURES_TIME.append((w, f'{var}-5GHZ-MIN', np.min(value_5GHZ)))
                FEATURES_TIME.append((w, f'{var}-5GHZ-STD', np.std(value_5GHZ)))
            else:
                FEATURES_TIME.append((w, f'{var}-5GHZ-MEAN', np.nan))
                FEATURES_TIME.append((w, f'{var}-5GHZ-MAX', np.nan))
                FEATURES_TIME.append((w, f'{var}-5GHZ-MIN', np.nan))
                FEATURES_TIME.append((w, f'{var}-5GHZ-STD', np.nan))
            if (value_2GHZ.size > 0):
                FEATURES_TIME.append((w, f'{var}-2GHZ-MEAN', np.mean(value_2GHZ)))
                FEATURES_TIME.append((w, f'{var}-2GHZ-MAX', np.max(value_2GHZ)))
                FEATURES_TIME.append((w, f'{var}-2GHZ-MIN', np.min(value_2GHZ)))
                FEATURES_TIME.append((w, f'{var}-2GHZ-STD', np.std(value_2GHZ)))
            else:
                FEATURES_TIME.append((w, f'{var}-2GHZ-MEAN', np.nan))
                FEATURES_TIME.append((w, f'{var}-2GHZ-MAX', np.nan))
                FEATURES_TIME.append((w, f'{var}-2GHZ-MIN', np.nan))
                FEATURES_TIME.append((w, f'{var}-2GHZ-STD', np.nan))

    # FEATURES_TIME looks like this <timestamp, feature name, value> 
    # (4000.0, 'Min-x', -1.848), (4000.0, 'Max-x', -1.659), (4000.0, 'Mean-x', -1.7706786155747838), (4000.0, 'Std-x', 0.023614236975760955)

    # Create a dataframe w/ column names: timestamps, feature, value  
    FEATURES_TIME = pd.DataFrame(FEATURES_TIME, columns=['timestamps', 'feature', 'value'])

    # Reshape data to produce a pivot table based on column values
    FEATURES_TIME = FEATURES_TIME.pivot(index='timestamps', columns='feature', values='value').reset_index()
    featured_rss.append(FEATURES_TIME)
    

In [0]:
featured_gps = []
WIN_SIZE_IN_MS = 1000 
OVERLAP_RATIO = 0.5 

for i in range(4):
    START_TIME, END_TIME = start[i], end[i]
    
    # arange([start, ]stop, [step, ]dtype=None)
    # find the end time of each window by considering the overlapping ratio 
    WINDOWS = np.arange(START_TIME + WIN_SIZE_IN_MS, END_TIME, WIN_SIZE_IN_MS * (1 - OVERLAP_RATIO))

    # new dataframe 
    FEATURES_TIME = []
    for w in WINDOWS:
        # for a given window, set the start and end time stamps 
        win_start, win_end = w - WIN_SIZE_IN_MS, w
      
        for var in ['accuracy', 'satellites']:
            # select the rows that belong to the current window, w
            value = GPS[i].loc[
                lambda x: (x['timeMillis'] >= win_start) & (x['timeMillis'] < win_end), var
            ].values

            # extract basic features 
            #min_v = np.min(value) # min
            #max_v = np.max(value) # max
            #mean_v = np.mean(value) # mean
            #std_v = np.std(value) # std. dev.

            # append each result (w: current window's end-timestamp, extracted feature) as a new row
            #FEATURES_TIME.append((w - , '{}-{}'.format('Min', var), min_v))
            #FEATURES_TIME.append((w, '{}-{}'.format('Max', var), max_v))
            #FEATURES_TIME.append((w, '{}-{}'.format('Mean', var), mean_v))
            #FEATURES_TIME.append((w, '{}-{}'.format('Std', var), std_v))
            if (value.size > 0):
                FEATURES_TIME.append((w, f'{var}', np.mean(value)))
            else:
                FEATURES_TIME.append((w, f'{var}', np.nan))

    # FEATURES_TIME looks like this <timestamp, feature name, value> 
    # (4000.0, 'Min-x', -1.848), (4000.0, 'Max-x', -1.659), (4000.0, 'Mean-x', -1.7706786155747838), (4000.0, 'Std-x', 0.023614236975760955)

    # Create a dataframe w/ column names: timestamps, feature, value  
    FEATURES_TIME = pd.DataFrame(FEATURES_TIME, columns=['timestamps', 'feature', 'value'])

    # Reshape data to produce a pivot table based on column values
    FEATURES_TIME = FEATURES_TIME.pivot(index='timestamps', columns='feature', values='value').reset_index()
    featured_gps.append(FEATURES_TIME)
    

In [0]:
WIN_SIZE_IN_MS = 1000 
OVERLAP_RATIO = 0.5
BIN_SIZE = 128
featured_acc = []

for i in range(4):
    START_TIME, END_TIME = start[i], end[i]
    DURATION_IN_SEC = (END_TIME - START_TIME) / 1000
    F_s = len(acc[i].index) / DURATION_IN_SEC
    P = 1.0 / F_s
    # arange([start, ]stop, [step, ]dtype=None)
    # find the end time of each window by considering the overlapping ratio 
    WINDOWS = np.arange(START_TIME + WIN_SIZE_IN_MS, END_TIME, WIN_SIZE_IN_MS * (1 - OVERLAP_RATIO))

    # new dataframe 
    FEATURES_FREQ = []
    for w in WINDOWS:
        # for a given window, set the start and end time stamps 
        win_start, win_end = w - WIN_SIZE_IN_MS, w
      
        for var in ['x', 'y', 'z']:
            # select the rows that belong to the current window, w
            value = acc[i].loc[
                lambda x: (x['timeMillis'] >= win_start) & (x['timeMillis'] < win_end), var
            ].values
            if (value.size > 0):
                value = ma(value)

                fft = np.fft.fft(value * np.hamming(value.shape[0]), n=BIN_SIZE)[1:BIN_SIZE//2]
                freq = np.fft.fftfreq(BIN_SIZE, P)[1:BIN_SIZE//2]
                amp = np.abs(fft)
                energy = amp ** 2
                amp_norm = amp / BIN_SIZE
                energy_norm = energy / BIN_SIZE

                freq_max_amp = freq[np.argmax(amp_norm)]
                weight_amp_avg = np.sum(amp * freq) / np.sum(amp)
                weight_energy_avg = np.sum(energy * freq) / np.sum(energy)
                power_entropy = - np.sum((energy / np.sum(energy)) * np.log(energy / np.sum(energy)))
                FEATURES_FREQ.append((w, '{}-{}'.format('Freq. Max. Amp', var), freq_max_amp))
                FEATURES_FREQ.append((w, '{}-{}'.format('Weighted Avg. Amp', var), weight_amp_avg))
                FEATURES_FREQ.append((w, '{}-{}'.format('Weighted Avg. Energy', var), weight_energy_avg))
                FEATURES_FREQ.append((w, '{}-{}'.format('Power Spec. Entropy', var), power_entropy))
            else:
                FEATURES_FREQ.append((w, '{}-{}'.format('Freq. Max. Amp', var), np.nan))
                FEATURES_FREQ.append((w, '{}-{}'.format('Weighted Avg. Amp', var), np.nan))
                FEATURES_FREQ.append((w, '{}-{}'.format('Weighted Avg. Energy', var), np.nan))
                FEATURES_FREQ.append((w, '{}-{}'.format('Power Spec. Entropy', var), np.nan))

    FEATURES_FREQ = pd.DataFrame(FEATURES_FREQ, columns=['timestamps', 'feature', 'value'])
    FEATURES_FREQ = FEATURES_FREQ.pivot(index='timestamps', columns='feature', values='value').reset_index()
    featured_acc.append(FEATURES_FREQ)

In [0]:
for i in range(4):
    featured_acc[i].set_index('timestamps', drop=True, inplace=True)
    featured_rss[i].set_index('timestamps', drop=True, inplace=True)
    featured_gps[i].set_index('timestamps', drop=True, inplace=True)

In [0]:
for i in range(4):
    pd.concat([featured_acc[i], featured_rss[i], featured_gps[i]], axis=1, sort=False).to_csv(f'{path}/output/Data{i}.csv')

In [0]:
Dataset = []
for i in range(4):
    Dataset.append(pd.read_csv(f'{path}/Data{i}.csv'))

In [0]:
label = []
for i in range(4):
    label.append(pd.read_csv(f'{path}/Transition{i}.csv'))

In [0]:
def floor_to_label(floor):
    return floor.map({
        -1: 'start',
        -5: 'MOI',
        -4: 'MII',
        -3: 'MIO',
        1: 'AOI',
        2: 'AII',
        3: 'AIO',
        6: 'ABOI',
        7: 'ABII',
        8: 'ABIO'
    })

In [0]:
data_interpolated = []
for i in range(4):
    data_interpolated.append(Dataset[i].interpolate(method='linear').fillna(method='bfill'))

In [0]:
print(pd.DataFrame(data_interpolated[3].isnull().any()))

                            0
timestamps              False
Freq. Max. Amp-x        False
Freq. Max. Amp-y        False
Freq. Max. Amp-z        False
Power Spec. Entropy-x   False
Power Spec. Entropy-y   False
Power Spec. Entropy-z   False
Weighted Avg. Amp-x     False
Weighted Avg. Amp-y     False
Weighted Avg. Amp-z     False
Weighted Avg. Energy-x  False
Weighted Avg. Energy-y  False
Weighted Avg. Energy-z  False
rss-2GHZ-MAX            False
rss-2GHZ-MEAN           False
rss-2GHZ-MIN            False
rss-2GHZ-STD            False
rss-5GHZ-MAX            False
rss-5GHZ-MEAN           False
rss-5GHZ-MIN            False
rss-5GHZ-STD            False
accuracy                False
satellites              False


In [0]:
from sklearn.preprocessing import MinMaxScaler
data_scaled = []
scaler = MinMaxScaler()
for i in range(4):
    data_scaled.append(scaler.fit_transform(data_interpolated[i]))

In [0]:
for i in range(4):
    print(data_interpolated[0].isnull().values.any())

False
False
False
False


In [0]:
import numpy as np

input_data = []
time_step = 10
for i in range(num_data):
    end = len(data_scaled[i]) - (len(data_scaled[i])%time_step)
    print(end)
    data_cut = data_scaled[i][:end]
    data_cut = pd.DataFrame(data_cut)
    for j in range(data_cut.shape[0] - time_step + 1):
        input_data.append(data_cut.loc[j:j+time_step-1].to_numpy())
    print(len(input_data))
    #input_data.append(np.reshape(data_cut, (-1, time_step, 23)))

6770
6761
2780
9532
5020
14543
5260
19794


In [0]:
for i in input_data:
    for j in i:
        INPUT.append(j)
len(INPUT)

217730

In [0]:
np.array(input_data).shape

(19794, 10, 23)

In [0]:
output_label = []

for i in range(num_data):
    end = len(data_scaled[i]) - (len(data_scaled[i])%time_step)
    for j in range(end):
        starting_point = data_interpolated[i]['timestamps'][j]
        ending_point = data_interpolated[i]['timestamps'][(j + time_step)]
        label_exist = label[i].loc[
            lambda x: (x['timeMillis'] >= starting_point) & (x['timeMillis'] < ending_point)
        ]['floor']
        if label_exist.size:
            add = False
            for val in label_exist.values:
                if val in [-5, -3, 1, 3, 6, 8]:
                    add = True
            if add:
                output_label.append(1)
        else:
            output_label.append(0)

In [0]:
print (INPUT[0])

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.11729285e-01 2.75799659e-01 4.41157048e-01 6.35987470e-02
  6.34899197e-01 4.37481127e-01 1.37751755e-01 1.08754400e-01
  2.82883070e-01 4.36619718e-01 3.14792899e-01 2.50000000e-01
  4.02827721e-01 1.23076923e-01 1.77429876e-01 2.27272727e-01
  1.53588678e-01 1.19718310e-01 8.92857143e-01]
 [1.47579703e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
  3.91069323e-01 2.94650100e-01 3.95982670e-01 1.68104645e-01
  2.94222333e-01 2.11367129e-01 1.40466432e-01 9.60509846e-02
  2.39350768e-01 4.22535211e-01 3.19028340e-01 2.44047619e-01
  3.93699297e-01 1.12087912e-01 1.65501817e-01 2.33766234e-01
  1.31647439e-01 1.19718310e-01 8.92857143e-01]
 [2.95159407e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
  3.83946164e-01 2.07121090e-01 1.03179496e-01 3.00233585e-01
  2.36155027e-01 6.16314389e-01 1.36555427e-01 5.75443425e-02
  8.20573930e-02 4.08450704e-01 3.23263781e-01 2.38095238e-01
  3.84570873e-01 1.01098901e-01 1.53

In [0]:
import random

X = np.array(input_data)
Y = np.array(output_label)
X_ = []
Y_ = []
print(Y.size)
for i in range(Y.size):
    if output_label[i] == 0:
        if (random.random() <= 0.1):
            X_.append(X[i])
            Y_.append(Y[i])
    else:
        X_.append(X[i])
        Y_.append(Y[i])
print(len(Y_))
print(len(X_))

19794
3053
3053


In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Flatten, LSTM, Dense, SimpleRNN, GRU
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Bidirectional

In [0]:
model_lstm = Sequential()
sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)
model_lstm.add(LSTM(10, activation='relu', input_shape=(10, 23)))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
history = model_lstm.fit(X_train, y_train, epochs=1000, validation_split=0.2, verbose=1)

Train on 1709 samples, validate on 428 samples
Epoch 1/1000
1709/1709 [==============================] - 5s 3ms/sample - loss: 0.6822 - acc: 0.6805 - val_loss: 0.6824 - val_acc: 0.6238
Epoch 2/1000
1709/1709 [==============================] - 1s 336us/sample - loss: 0.6741 - acc: 0.6524 - val_loss: 0.6795 - val_acc: 0.5467
Epoch 3/1000
1709/1709 [==============================] - 1s 319us/sample - loss: 0.6676 - acc: 0.6267 - val_loss: 0.6775 - val_acc: 0.5584
Epoch 4/1000
1709/1709 [==============================] - 1s 312us/sample - loss: 0.6625 - acc: 0.6273 - val_loss: 0.6762 - val_acc: 0.5584
Epoch 5/1000
1709/1709 [==============================] - 1s 357us/sample - loss: 0.6582 - acc: 0.6273 - val_loss: 0.6749 - val_acc: 0.5584
Epoch 6/1000
1709/1709 [==============================] - 1s 358us/sample - loss: 0.6548 - acc: 0.6273 - val_loss: 0.6739 - val_acc: 0.5584
Epoch 7/1000
1709/1709 [==============================] - 1s 325us/sample - loss: 0.6518 - acc: 0.6273 - val_loss: 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_), np.array(Y_), test_size=0.3, random_state=42)

In [0]:
model.evaluate(X_test, y_test)

916/916 [==============================] - 0s 92us/sample - loss: 0.5349 - acc: 0.7162


[0.534866997508503, 0.7161572]

In [0]:
Xnew = model.predict(X_test)
Xnew

array([[7.55049884e-01],
       [0.00000000e+00],
       [2.52921045e-01],
       [2.22662091e-03],
       [5.68662345e-01],
       [5.14663517e-01],
       [3.71919274e-02],
       [4.74198014e-01],
       [1.44782662e-03],
       [4.73789096e-01],
       [5.22073984e-01],
       [1.45494938e-03],
       [3.93265575e-01],
       [5.70976257e-01],
       [8.52626562e-02],
       [3.84563208e-03],
       [1.07914209e-04],
       [1.20345652e-01],
       [1.02519989e-04],
       [1.35108113e-01],
       [9.00179148e-04],
       [2.07251072e-01],
       [1.51360035e-01],
       [1.73854649e-01],
       [1.77025795e-05],
       [6.79333925e-01],
       [1.07531130e-01],
       [1.18243694e-02],
       [4.40116525e-01],
       [2.44959474e-01],
       [0.00000000e+00],
       [9.38499391e-01],
       [4.54572320e-01],
       [2.64671803e-01],
       [5.56396544e-01],
       [2.68690288e-01],
       [1.64159834e-02],
       [3.86865675e-01],
       [2.03925550e-01],
       [1.36585236e-02],


In [0]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,

array([[[0.85891381, 0.        , 0.        , ..., 0.38291609,
         0.00704225, 1.        ],
        [0.85906139, 0.        , 0.        , ..., 0.37264862,
         0.00704225, 1.        ],
        [0.85920897, 0.        , 0.        , ..., 0.36238115,
         0.00704225, 1.        ],
        ...,
        [0.85994687, 0.        , 0.        , ..., 0.39144459,
         0.00704225, 0.96428571],
        [0.86009445, 0.        , 0.        , ..., 0.41624378,
         0.00704225, 0.96428571],
        [0.86024203, 0.        , 0.        , ..., 0.44104297,
         0.00704225, 0.96428571]],

       [[0.9559103 , 0.5       , 0.        , ..., 0.54541267,
         0.20634921, 0.7037037 ],
        [0.95610034, 0.25      , 0.        , ..., 0.58435396,
         0.20634921, 0.7037037 ],
        [0.95629038, 0.        , 0.        , ..., 0.58435396,
         0.20634921, 0.7037037 ],
        ...,
        [0.95724059, 0.        , 0.        , ..., 0.62471056,
         0.20634921, 0.7037037 ],
        [0.9

In [0]:
model_rnn = Sequential()
sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)
model_rnn.add(SimpleRNN(10, activation='relu', input_shape=(10, 23)))
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
history = model_rnn.fit(X_train, y_train, epochs=1000, validation_split=0.2, verbose=1)

Train on 1709 samples, validate on 428 samples
Epoch 1/1000
1709/1709 [==============================] - 5s 3ms/sample - loss: 0.6651 - acc: 0.6080 - val_loss: 0.7018 - val_acc: 0.5537
Epoch 2/1000
1709/1709 [==============================] - 0s 122us/sample - loss: 0.6508 - acc: 0.6220 - val_loss: 0.7021 - val_acc: 0.5584
Epoch 3/1000
1709/1709 [==============================] - 0s 120us/sample - loss: 0.6402 - acc: 0.6243 - val_loss: 0.7006 - val_acc: 0.5584
Epoch 4/1000
1709/1709 [==============================] - 0s 126us/sample - loss: 0.6299 - acc: 0.6255 - val_loss: 0.6854 - val_acc: 0.5584
Epoch 5/1000
1709/1709 [==============================] - 0s 135us/sample - loss: 0.6203 - acc: 0.6226 - val_loss: 0.6815 - val_acc: 0.5584
Epoch 6/1000
1709/1709 [==============================] - 0s 121us/sample - loss: 0.6117 - acc: 0.6191 - val_loss: 0.6779 - val_acc: 0.5607
Epoch 7/1000
1709/1709 [==============================] - 0s 122us/sample - loss: 0.6039 - acc: 0.6161 - val_loss: 

In [0]:
model_rnn.evaluate(X_test, y_test)

916/916 [==============================] - 0s 71us/sample - loss: 0.5857 - acc: 0.6943


[0.5856988521121995, 0.6943231]

In [0]:
model_lstm.evaluate(X_test, y_test)

916/916 [==============================] - 0s 106us/sample - loss: 0.5657 - acc: 0.6921


[0.5657174898287094, 0.69213974]

In [0]:
model_gru.evaluate(X_test, y_test)

NameError: ignored

In [0]:
model_gru = Sequential()
sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)
model_gru.add(GRU(10, activation='relu', input_shape=(10, 23)))
model_gru.add(Dense(1, activation='sigmoid'))
model_gru.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
history = model_gru.fit(X_train, y_train, epochs=1000, validation_split=0.2, verbose=1)

Train on 1709 samples, validate on 428 samples
Epoch 1/1000
1709/1709 [==============================] - 6s 3ms/sample - loss: 0.6761 - acc: 0.5524 - val_loss: 0.6888 - val_acc: 0.5280
Epoch 2/1000
1709/1709 [==============================] - 1s 296us/sample - loss: 0.6629 - acc: 0.6097 - val_loss: 0.6904 - val_acc: 0.5514
Epoch 3/1000
1709/1709 [==============================] - 1s 296us/sample - loss: 0.6575 - acc: 0.6232 - val_loss: 0.6917 - val_acc: 0.5561
Epoch 4/1000
1709/1709 [==============================] - 1s 301us/sample - loss: 0.6538 - acc: 0.6249 - val_loss: 0.6905 - val_acc: 0.5561
Epoch 5/1000
1709/1709 [==============================] - 1s 314us/sample - loss: 0.6508 - acc: 0.6267 - val_loss: 0.6899 - val_acc: 0.5584
Epoch 6/1000
1709/1709 [==============================] - 1s 299us/sample - loss: 0.6481 - acc: 0.6267 - val_loss: 0.6870 - val_acc: 0.5584
Epoch 7/1000
1709/1709 [==============================] - 1s 302us/sample - loss: 0.6453 - acc: 0.6273 - val_loss: 

In [0]:
model_stacked_lstm = Sequential()
sgd = optimizers.SGD(lr=0.001, clipvalue=0.5)
model_stacked_lstm.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(10, 23)))
model_stacked_lstm.add(LSTM(100, activation='relu', return_sequences=True))
model_stacked_lstm.add(LSTM(50, activation='relu'))
model_stacked_lstm.add(Dense(1, activation='sigmoid'))
model_stacked_lstm.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
history = model_stacked_lstm.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=1)

Train on 1709 samples, validate on 428 samples
Epoch 1/100
1709/1709 [==============================] - 10s 6ms/sample - loss: 0.6943 - acc: 0.4219 - val_loss: 0.6935 - val_acc: 0.4720
Epoch 2/100
1709/1709 [==============================] - 5s 3ms/sample - loss: 0.6928 - acc: 0.5202 - val_loss: 0.6928 - val_acc: 0.5584
Epoch 3/100
1709/1709 [==============================] - 5s 3ms/sample - loss: 0.6913 - acc: 0.6261 - val_loss: 0.6921 - val_acc: 0.5584
Epoch 4/100
1709/1709 [==============================] - 5s 3ms/sample - loss: 0.6898 - acc: 0.6273 - val_loss: 0.6915 - val_acc: 0.5584
Epoch 5/100
1709/1709 [==============================] - 5s 3ms/sample - loss: 0.6885 - acc: 0.6273 - val_loss: 0.6910 - val_acc: 0.5584
Epoch 6/100
1709/1709 [==============================] - 5s 3ms/sample - loss: 0.6871 - acc: 0.6273 - val_loss: 0.6905 - val_acc: 0.5584
Epoch 7/100
1709/1709 [==============================] - 5s 3ms/sample - loss: 0.6859 - acc: 0.6273 - val_loss: 0.6900 - val_acc: 

KeyboardInterrupt: ignored